In [1]:
using EtherArrays
using StaticArrays
using KernelAbstractions
using oneAPI

In [2]:
struct Par{N, T}
    mass::StaticScalar{T}
    rho::StaticScalar{T}
    pos::StaticVector{N, T}
    vel::StaticVector{N, T}
    acc::StaticVector{N, T}
    stress::StaticMatrix{N, N, T}
end

In [3]:
const kBackend = oneAPI.oneAPIBackend()
const Tcontainer = oneAPI.oneArray
const Tf = Float32

Float32

In [9]:
cpu_struct = [
    Par{2, Tf}(
        Scalar{Tf}(1.0),
        Scalar{Tf}(1.0),
        SVector{2, Tf}(1.0, 2.0),
        SVector{2, Tf}(3.0, 4.0),
        SVector{2, Tf}(5.0, 6.0),
        SMatrix{2, 2, Tf}(7.0, 8.0, 9.0, 10.0)
    )
    for _ in 1:3
]

3-element Vector{Par{2, Float32}}:
 Par{2, Float32}(fill(1.0f0), fill(1.0f0), Float32[1.0, 2.0], Float32[3.0, 4.0], Float32[5.0, 6.0], Float32[7.0 9.0; 8.0 10.0])
 Par{2, Float32}(fill(1.0f0), fill(1.0f0), Float32[1.0, 2.0], Float32[3.0, 4.0], Float32[5.0, 6.0], Float32[7.0 9.0; 8.0 10.0])
 Par{2, Float32}(fill(1.0f0), fill(1.0f0), Float32[1.0, 2.0], Float32[3.0, 4.0], Float32[5.0, 6.0], Float32[7.0 9.0; 8.0 10.0])

In [10]:
gpu_struct = oneAPI.oneArray(cpu_struct)

3-element oneArray{Par{2, Float32}, 1, oneAPI.oneL0.DeviceBuffer}:
 Par{2, Float32}(fill(1.0f0), fill(1.0f0), Float32[1.0, 2.0], Float32[3.0, 4.0], Float32[5.0, 6.0], Float32[7.0 9.0; 8.0 10.0])
 Par{2, Float32}(fill(1.0f0), fill(1.0f0), Float32[1.0, 2.0], Float32[3.0, 4.0], Float32[5.0, 6.0], Float32[7.0 9.0; 8.0 10.0])
 Par{2, Float32}(fill(1.0f0), fill(1.0f0), Float32[1.0, 2.0], Float32[3.0, 4.0], Float32[5.0, 6.0], Float32[7.0 9.0; 8.0 10.0])

In [13]:
@kernel function ker!(gpu_s)
    idx = @index(Global)
    gpu_s[idx].pos += gpu_s[idx].vel
end

In [14]:
ker!(oneAPIBackend(), 3)(gpu_struct, ndrange=(3,))

GPUCompiler.InvalidIRError: InvalidIRError: compiling MethodInstance for gpu_ker!(::KernelAbstractions.CompilerMetadata{KernelAbstractions.NDIteration.DynamicSize, KernelAbstractions.NDIteration.DynamicCheck, Nothing, CartesianIndices{1, Tuple{Base.OneTo{Int64}}}, KernelAbstractions.NDIteration.NDRange{1, KernelAbstractions.NDIteration.DynamicSize, KernelAbstractions.NDIteration.StaticSize{(3,)}, CartesianIndices{1, Tuple{Base.OneTo{Int64}}}, Nothing}}, ::oneDeviceVector{Par{2, Float32}, 1}) resulted in invalid LLVM IR
Reason: unsupported call to an unknown function (call to gpu_malloc)
Stacktrace:
 [1] malloc
   @ ~/.julia/packages/GPUCompiler/j4HFa/src/runtime.jl:85
 [2] gc_pool_alloc
   @ ~/.julia/packages/GPUCompiler/j4HFa/src/runtime.jl:116
 [3] gpu_ker!
   @ ~/.julia/packages/KernelAbstractions/X5fk1/src/macros.jl:332
 [4] gpu_ker!
   @ ./none:0
Reason: unsupported dynamic function invocation (call to _map)
Stacktrace:
 [1] map
   @ ~/.julia/packages/StaticArrays/DsPgf/src/mapred